In [1]:
import os
import pandas as pd
import scanpy as sc
import os
import numpy as np
import anndata as ad
from anndata import AnnData
from anndata import read_h5ad
import gc  

DATASET_DIR = "/work/upcourtine/clock-classifier/gabriele-results/census_results/datasets"

# Make GeneMapping File

Raw data from [here](https://www.gencodegenes.org/human/).

Unzip the file `gunzip gencode.v47.chr_patch_hapl_scaff.annotation.gtf.gz`.

In [ ]:
import pandas as pd
import re

# Define chunk size
chunk_size = 100000

# Initialize an empty list to store chunks
chunks = []

# Read the GTF file in chunks
for chunk in pd.read_csv("/home/dalai/git/clock-classifier/Gabriele/assets/gencode.v47.chr_patch_hapl_scaff.annotation.gtf", 
                         sep="\t", 
                         comment="#", 
                         header=None, 
                         names=["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attributes"], 
                         chunksize=chunk_size):
    
    # Extract gene_id, transcript_id, gene_name, and gene_type from the 'attributes' column using regular expressions
    chunk['gene_id'] = chunk['attributes'].apply(lambda x: re.search(r'gene_id "([^"]+)"', x).group(1) if re.search(r'gene_id "([^"]+)"', x) else None)
    chunk['transcript_id'] = chunk['attributes'].apply(lambda x: re.search(r'transcript_id "([^"]+)"', x).group(1) if re.search(r'transcript_id "([^"]+)"', x) else None)
    chunk['gene_name'] = chunk['attributes'].apply(lambda x: re.search(r'gene_name "([^"]+)"', x).group(1) if re.search(r'gene_name "([^"]+)"', x) else None)
    chunk['gene_type'] = chunk['attributes'].apply(lambda x: re.search(r'gene_type "([^"]+)"', x).group(1) if re.search(r'gene_type "([^"]+)"', x) else None)

    # Add the chunk to the list
    chunks.append(chunk)

# Combine the chunks into a single DataFrame
df = pd.concat(chunks, ignore_index=True)

# Display the first few rows with gene_id, transcript_id, gene_name, and gene_type
display(df[['seqname', 'feature', 'start', 'end', 'gene_id', 'transcript_id', 'gene_name', 'gene_type']].head())


In [ ]:
print(df.feature.unique())
print(df.gene_type.unique())

In [ ]:
# Filter

print(df.shape)
print(df[df.feature == "gene"].shape)

# Take only elements that are genes
print(df.feature.unique())
df = df[df.feature == "gene"]

# Maybe filter more based on "gene_type" ??
# TODO

df.head(4)

In [ ]:
# Take only mapping colums
df = df[["gene_id", "gene_name"]]
df.head(5)

In [ ]:
# Remove "version" from ensamble id

print(df['gene_id'].str.split('.').str[0].size) #beforee
print(df['gene_id'].str.split('.').str[0].unique().size) #see if thwere are some duplicates after removing version

df['gene_id'] = df['gene_id'].str.split('.').str[0]


In [ ]:
# Save
save_mapping_df_path = os.path.join("/home/dalai/git/clock-classifier/Gabriele/assets", "mapping_genes_census.csv")
df.to_csv(save_mapping_df_path, index=False)


In [ ]:
# little chck
df = pd.read_csv(save_mapping_df_path, index_col = False)
df